<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Start-to-Finish Example: $\text{GiRaFFE_HO}$ 1D tests

### Author: Patrick Nelson

### Adapted from [Start-to-Finish Example: Head-On Black Hole Collision](Tutorial-Start_to_Finish-BSSNCurvilinear-Two_BHs_Collide.ipynb)

## This module implements a basic GRFFE code to evolve one-dimensional GRFFE waves.

### NRPy+ Source Code for this module: 
1. [GiRaFFEfood_HO/GiRaFFEfood_HO_1D_tests.py](../edit/GiRaFFEfood_HO/GiRaFFEfood_HO_1D_tests.py); [\[**tutorial**\]](Tutorial-GiRaFFEfood_HO_1D_tests.ipynb): Aligned rotator initial data, sets all FFE variables in a Cartesian basis.
1. [GiRaFFE_HO/GiRaFFE_Higher_Order_v2.py](../edit/GiRaFFE_HO/GiRaFFE_Higher_Order_v2.py); [\[**tutorial**\]](Tutorial-GiRaFFE_Higher_Order_v2.ipynb): Generates the right-hand sides for the GRFFE evolution equations in Cartesian coordinates.
We will also borrow C code from the ETK implementation of $\text{GiRaFFE_HO}$

Here we use NRPy+ to generate the C source code necessary to set up initial data for a model neutron star (see [the original GiRaFFE paper](https://arxiv.org/pdf/1704.00599.pdf)). Then we use it to generate the RHS expressions for [Method of Lines](https://reference.wolfram.com/language/tutorial/NDSolveMethodOfLines.html) time integration based on the [explicit Runge-Kutta fourth-order scheme](https://en.wikipedia.org/wiki/Runge%E2%80%93Kutta_methods) (RK4).

The entire algorithm is outlined below, with NRPy+-based components highlighted in <font color='green'>green</font>.

1. Allocate memory for gridfunctions, including temporary storage for the RK4 time integration.
1. (**Step 2** below) <font color='green'>Set gridfunction values to initial data (**[documented in previous module](Tutorial-GiRaFFEfood_HO_1D_tests.ipynb)**).</font>
1. Evolve the initial data forward in time using RK4 time integration. At each RK4 substep, do the following:
    1. (**Step 3A** below) <font color='green'>Evaluate GRFFE RHS expressions.</font>
    1. (**Step 4** below) Apply singular, curvilinear coordinate boundary conditions [*a la* the SENR/NRPy+ paper](https://arxiv.org/abs/1712.07658)
1. (**Step 3B** below) At the end of each iteration in time, output the <font color='green'>FFE variables</font>. (This is in Step 3B, because Step 4 requires that *all* gridfunctions be defined.)
1. Repeat above steps at two numerical resolutions to confirm convergence to the expected value.

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

1. [Step 1](#writec): Use NRPy+ to generate necessary C code to solve the scalar wave equation in curvilinear, singular coordinates
    1. [Step 1.a](#mol_timestep): Use NRPy+ to generate the RK4 timestepping method
    1. [Step 1.b](#c_headers): Use NRPy+ to generate C headers
    1. [Step 1.c](#copy_c): Copy code over from the GiRaFFE C code library
    1. [Step 1.d](#initial_data): Import Alfv&eacute;n Wave initial data C function
    1. [Step 1.e](#poynting): Import densitized Poynting flux initial data conversion C function
    1. [Step 1.f](#rhs): Output GiRaFFE RHS expressions
1. [Step 2](#mainc): `GiRaFFE_standalone.c`: The Main C Code
    1. [Step 2.a](#import_headers): Import needed header files
    1. [Step 2.b](#data_type): Set data type
    1. [Step 2.c](#free_params): Set free parameters
    1. [Step 2.d](#idx4): Declare the IDX4 macro
    1. [Step 2.e](#gridfuncs): Define gridfunctions
    1. [Step 2.f](#bcs): Boundary Conditions, the A-to-B driver, and the conservative-to-primitive solver
    1. [Step 2.g](#timestep): Find the CFL-constrained timestep
    1. [Step 2.h](#initial_data_c): Declare the function for the exact solution
    1. [Step 2.i](#rhsC): Declare the functions to evaluate the GRFFE RHSs
    1. [Step 2.j](#main): The `main()` function
1. [Step 3](#convergence): Code validation: Verify that relative error in numerical solution converges to zero at the expected order
1. [Step 4](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file


<a id='writec'></a>

# Step 1: Use NRPy+ to generate necessary C code to solve the scalar wave equation in curvilinear, singular coordinates \[Back to [top](#toc)\]
$$\label{writec}$$

We first begin by importing the needed NRPy+ modules; we then create the directory to which we wish to write our C code. Note that we first remove it and all its contents to make sure we have a fresh start. We set the FD-order to two and our spatial dimension to 3. We set our coordinate system to Cartesian and set up the reference metric. 

We also set the symmetry axes parameter from `indexedexp` to `2`. This is done because the Alfv&eacute;n wave is completely independent of $z$, which is direction `2` in Cartesian coordinates; so, we can speed up the code in this case by assuming that *any* derivative in the $z$ direction is 0.

In [1]:
import os,sys
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

# Step P1: First we import needed core NRPy+ modules
from outputC import *            # NRPy+: Core C code output module
import finite_difference as fin  # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import grid as gri               # NRPy+: Functions having to do with numerical grids
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm   # NRPy+: Reference metric support

#Step P2: Create C code output directory:
import cmdline_helper as cmd
import shutil, os
cmd.delete_existing_files("GiRaFFE_standalone_Ccodes/*")
Ccodesdir = os.path.join("GiRaFFE_standalone_Ccodes/")
cmd.mkdir(Ccodesdir)

# Set the finite differencing order to 4.
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 2)

# Set spatial dimension (must be 3 for BSSN)
DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

# Then we set the coordinate system for the numerical grid
par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
rfm.reference_metric() # Create ReU, ReDD needed for rescaling B-L initial data, generating BSSN RHSs, etc.

# Then we set the y and z axes to be the symmetry axis; i.e., axis "2", corresponding to the i2 direction.
#      This sets all spatial derivatives in the phi direction to zero.
par.set_parval_from_str("indexedexp::symmetry_axes","2")

# Then we set the phi axis to be the symmetry axis; i.e., axis "2", corresponding to the i2 direction.
#      This sets all spatial derivatives in the phi direction to zero.
#par.set_parval_from_str("indexedexp::symmetry_axes","2") # Let's not deal with this yet.



<a id='mol_timestep'></a>

## Step 1.a: Use NRPy+ to generate the RK4 timestepping method \[Back to [top](#toc)\]
$$\label{mol_timestep}$$

Next, we will we will use the `MoLtimestepping` module to write the code for our timestepping algorithm. Here, we will use it to write RK4, but note that by changing the variable `RK_method` we can easily and immediately use many other algorithms. 

It is also imperative to pass the correct strings `RHS_string` and `post_RHS_string`. *Any* functions that we want to call between each step need to be included here.

In [2]:
# Choices are: Euler, "RK2 Heun", "RK2 MP", "RK2 Ralston", RK3, "RK3 Heun", "RK3 Ralston",
#              SSPRK3, RK4, DP5, DP5alt, CK5, DP6, L6, DP8
RK_method = "RK4"

# Generate timestepping code. As described above the Table of Contents, this is a 3-step process:
#       3.A: Evaluate RHSs (RHS_string)
#       3.B: Apply boundary conditions (post_RHS_string, pt 1)
#       3.C: Enforce det(gammabar) = det(gammahat) constraint (post_RHS_string, pt 2)
import MoLtimestepping.C_Code_Generation as MoL
from MoLtimestepping.RK_Butcher_Table_Dictionary import Butcher_dict
RK_order  = Butcher_dict[RK_method][1]
cmd.mkdir(os.path.join(Ccodesdir,"MoLtimestepping/"))
MoL.MoL_C_Code_Generation(RK_method,
    RHS_string = """
calc_u0(Nxx_plus_2NGHOSTS,aux_gfs);
quantities_to_FD_for_rhs_eval(Nxx_plus_2NGHOSTS,dxx,xx,RK_INPUT_GFS,aux_gfs);
rhs_eval(Nxx,Nxx_plus_2NGHOSTS,dxx, xx, RK_INPUT_GFS, aux_gfs, RK_OUTPUT_GFS);""",
    post_RHS_string = """
GiRaFFE_HO_conserv_to_prims_FFE(Nxx, Nxx_plus_2NGHOSTS, dxx,xx, RK_OUTPUT_GFS, aux_gfs);
apply_bcs(Nxx, Nxx_plus_2NGHOSTS, RK_OUTPUT_GFS, aux_gfs);
driver_A_to_B(Nxx, Nxx_plus_2NGHOSTS, dxx, RK_OUTPUT_GFS, aux_gfs);
//apply_bcs_EXACT(Nxx,Nxx_plus_2NGHOSTS,xx,n,dt,RK_OUTPUT_GFS,aux_gfs);\n""",
    outdir = os.path.join(Ccodesdir,"MoLtimestepping/"))


<a id='c_headers'></a>

## Step 1.b: Use NRPy+ to generate C headers \[Back to [top](#toc)\]
$$\label{c_headers}$$

We will also need to output C header files that are related to the numerical grids and the coordinate system we set up using the reference metric. These end up as very simple files in Cartesian, but they will be more complex in other coordinate systems.

In [3]:
#################
# Next output C headers related to the numerical grids we just set up:
#################

# First output the coordinate bounds xxmin[] and xxmax[]:
with open(os.path.join(Ccodesdir,"xxminmax.h"), "w") as file:
    file.write("const REAL xxmin[3] = {"+str(rfm.xxmin[0])+","+str(rfm.xxmin[1])+","+str(rfm.xxmin[2])+"};\n")
    file.write("const REAL xxmax[3] = {"+str(rfm.xxmax[0])+","+str(rfm.xxmax[1])+","+str(rfm.xxmax[2])+"};\n")

# Next output the proper distance between gridpoints in given coordinate system.
#     This is used to find the minimum timestep.
dxx     = ixp.declarerank1("dxx",DIM=3)
ds_dirn = rfm.ds_dirn(dxx)
outputC([ds_dirn[0],ds_dirn[1],ds_dirn[2]],["ds_dirn0","ds_dirn1","ds_dirn2"],os.path.join(Ccodesdir,"ds_dirn.h"))

# Generic coordinate NRPy+ file output, Part 2: output the conversion from (x0,x1,x2) to Cartesian (x,y,z)
outputC([rfm.xx_to_Cart[0],rfm.xx_to_Cart[1],rfm.xx_to_Cart[2]],["xCart[0]","xCart[1]","xCart[2]"],
        os.path.join(Ccodesdir,"xx_to_Cart.h"))

Wrote to file "GiRaFFE_standalone_Ccodes/ds_dirn.h"
Wrote to file "GiRaFFE_standalone_Ccodes/xx_to_Cart.h"


<a id='copy_c'></a>

## Step 1.c: Copy code over from the GiRaFFE C code library \[Back to [top](#toc)\]
$$\label{copy_c}$$

There are some important C codes that we have already written that are stored elsewhere. We will now copy them to our working directory. More detail about these codes can be found here:
* [Tutorial-GiRaFFE_HO_C_code_library-A2B.ipynb](Tutorial-GiRaFFE_HO_C_code_library-A2B.ipynb)
* [Tutorial-GiRaFFE_HO_C_code_library-C2P_P2C.ipynb](Tutorial-GiRaFFE_HO_C_code_library-C2P_P2C.ipynb)
* [Tutorial-GiRaFFE_HO_C_code_library-BCs.ipynb](Tutorial-GiRaFFE_HO_C_code_library-BCs.ipynb)

In [4]:
# First, let's make sure that the directories to which we want to copy files exist.
# The cmdline_helper function mkdir will do this regardless of OS

cmd.mkdir(os.path.join(Ccodesdir,"boundary_conditions/"))
cmd.mkdir(os.path.join(Ccodesdir,"A2B/"))

# Now, we'll start copying files using shutil.copy(src,dst)
shutil.copy(os.path.join("GiRaFFE_HO/GiRaFFE_Ccode_library/A2B/driver_AtoB.c"),os.path.join(Ccodesdir,"A2B/"))

shutil.copy(os.path.join("GiRaFFE_HO/GiRaFFE_Ccode_library/driver_conserv_to_prims_FFE.C"),os.path.join(Ccodesdir))
shutil.copy(os.path.join("GiRaFFE_HO/GiRaFFE_Ccode_library/compute_conservatives_FFE.C"),os.path.join(Ccodesdir))

shutil.copy(os.path.join("GiRaFFE_HO/GiRaFFE_Ccode_library/boundary_conditions/GiRaFFE_boundary_conditions.h"),os.path.join(Ccodesdir,"boundary_conditions/"))


'GiRaFFE_standalone_Ccodes/boundary_conditions/GiRaFFE_boundary_conditions.h'

<a id='initial_data'></a>

## Step 1.d: Import Alfv&eacute;n Wave initial data C function
$$\label{initial_data}$$

The [GiRaFFEfood_HO.GiRaFFEfood_HO_1D_tests.py](../edit/GiRaFFEfood_HO/GiRaFFEfood_HO_AlignedRotator.py) NRPy+ module does the following:

1. Set up Alfv&eacute;n Wave initial data quantities in the **Cartesian basis**, as [documented here](Tutorial-GiRaFFEfood_HO_1D_tests.ipynb). 


In [5]:
import GiRaFFEfood_HO.GiRaFFEfood_HO_1D_tests as gf1D
gf1D.GiRaFFEfood_HO_1D_tests()

# Step 2: Create the C code output kernel.
#BU = ixp.register_gridfunctions_for_single_rank1("AUX","BU")
GiRaFFEfood_A_v_to_print_left   = [\
                                   lhrh(lhs=gri.gfaccess("out_gfs","AD0"),rhs=gf1D.AleftD[0]),\
                                   lhrh(lhs=gri.gfaccess("out_gfs","AD1"),rhs=gf1D.AleftD[1]),\
                                   lhrh(lhs=gri.gfaccess("out_gfs","AD2"),rhs=gf1D.AleftD[2]),\
                                   lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU0"),rhs=gf1D.ValenciavleftU[0]),\
                                   lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU1"),rhs=gf1D.ValenciavleftU[1]),\
                                   lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU2"),rhs=gf1D.ValenciavleftU[2]),\
                                  ]
GiRaFFEfood_A_v_to_print_center = [\
                                   lhrh(lhs=gri.gfaccess("out_gfs","AD0"),rhs=gf1D.AcenterD[0]),\
                                   lhrh(lhs=gri.gfaccess("out_gfs","AD1"),rhs=gf1D.AcenterD[1]),\
                                   lhrh(lhs=gri.gfaccess("out_gfs","AD2"),rhs=gf1D.AcenterD[2]),\
                                   lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU0"),rhs=gf1D.ValenciavcenterU[0]),\
                                   lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU1"),rhs=gf1D.ValenciavcenterU[1]),\
                                   lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU2"),rhs=gf1D.ValenciavcenterU[2]),\
                                  ]
GiRaFFEfood_A_v_to_print_right  = [\
                                   lhrh(lhs=gri.gfaccess("out_gfs","AD0"),rhs=gf1D.ArightD[0]),\
                                   lhrh(lhs=gri.gfaccess("out_gfs","AD1"),rhs=gf1D.ArightD[1]),\
                                   lhrh(lhs=gri.gfaccess("out_gfs","AD2"),rhs=gf1D.ArightD[2]),\
                                   lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU0"),rhs=gf1D.ValenciavrightU[0]),\
                                   lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU1"),rhs=gf1D.ValenciavrightU[1]),\
                                   lhrh(lhs=gri.gfaccess("out_gfs","ValenciavU2"),rhs=gf1D.ValenciavrightU[2]),\
                                  ]

GiRaFFEfood_A_v_CKernel_left   = fin.FD_outputC("returnstring",GiRaFFEfood_A_v_to_print_left,   params="outCverbose=False")
GiRaFFEfood_A_v_CKernel_center = fin.FD_outputC("returnstring",GiRaFFEfood_A_v_to_print_center, params="outCverbose=False")
GiRaFFEfood_A_v_CKernel_right  = fin.FD_outputC("returnstring",GiRaFFEfood_A_v_to_print_right,  params="outCverbose=False")

# Step 4: Write the C code kernel to file.
with open(os.path.join(Ccodesdir,"GiRaFFEfood_A_v_1D_tests_left.h"), "w") as file:
    file.write(str(GiRaFFEfood_A_v_CKernel_left).replace("IDX4","IDX4S"))

with open(os.path.join(Ccodesdir,"GiRaFFEfood_A_v_1D_tests_center.h"), "w") as file:
    file.write(str(GiRaFFEfood_A_v_CKernel_center).replace("IDX4","IDX4S"))

with open(os.path.join(Ccodesdir,"GiRaFFEfood_A_v_1D_tests_right.h"), "w") as file:
    file.write(str(GiRaFFEfood_A_v_CKernel_right).replace("IDX4","IDX4S"))

# We will also need to declare some C parameters for this initial data
lbound,rbound = par.Cparameters("REAL","GiRaFFEfood_HO_1D",["lbound","rbound"], [-0.1,0.1])

<a id='poynting'></a>

## Step 1.e: Import densitized Poynting flux initial data conversion C function
$$\label{poynting}$$

The [GiRaFFEfood_HO.GiRaFFEfood_HO.py](../edit/GiRaFFEfood_HO/GiRaFFEfood_HO.py) NRPy+ module does the following:

1. Set up Exact Wald initial data quantities in the **Cartesian basis**, as [documented here](Tutorial-GiRaFFEfood_HO_Aligned_Rotator.ipynb).
2. Convert initial magnetic fields and Valencia 3-velocities into densitized Poynting flux initial data.

We only use the second functionality here (for now).


In [6]:
# Step 2: Create the C code output kernel.
gri.glb_gridfcs_list = []
import GiRaFFEfood_HO.GiRaFFEfood_HO_Exact_Wald as gfho
gfho.GiRaFFEfood_HO_ID_converter()
# To best format this for the ETK, we'll need to register this gridfunction.
StildeD = ixp.register_gridfunctions_for_single_rank1("EVOL","StildeD")
GiRaFFE_S_to_print = [\
                      lhrh(lhs=gri.gfaccess("out_gfs","StildeD0"),rhs=gfho.StildeD[0]),\
                      lhrh(lhs=gri.gfaccess("out_gfs","StildeD1"),rhs=gfho.StildeD[1]),\
                      lhrh(lhs=gri.gfaccess("out_gfs","StildeD2"),rhs=gfho.StildeD[2]),\
                     ]

GiRaFFE_S_CKernel = fin.FD_outputC("returnstring",GiRaFFE_S_to_print,params="outCverbose=False")

# Format the code within a C loop over cctkGH
#GiRaFFE_S_looped = lp.loop(["i2","i1","i0"],["0","0","0"],
#                           ["Nxx_plus_2NGHOSTS[2]","Nxx_plus_2NGHOSTS[1]","Nxx_plus_2NGHOSTS[0]"],\
#                           ["1","1","1"],["#pragma omp parallel for","",""],"",\
#                           GiRaFFE_S_CKernel.replace("time","cctk_time"))

# Step 4: Write the C code kernel to file.
with open(os.path.join(Ccodesdir,"GiRaFFEfood_HO_Stilde.h"), "w") as file:
    file.write(str(GiRaFFE_S_CKernel))


<a id='rhs'></a>

## Step 1.f: Output GiRaFFE RHS expressions
$$\label{rhs}$$


In [7]:
gri.glb_gridfcs_list = [] # This is necessary because, since this was originally designed as two ETK thorns,
                          # some gridfunctions are registered twice.

import GiRaFFE_HO.GiRaFFE_Higher_Order_v2 as gho
gho.GiRaFFE_Higher_Order_v2()

# Declaring StildeD as a gridfunction is unnecessary in GiRaFFE_HO. While it was declared in GiRaFFEfood_HO,
# those have since been cleared to avoid conflict; so, we re-declare it here.
StildeD = ixp.register_gridfunctions_for_single_rank1("EVOL","StildeD")

# Create the C code output kernel.
# Here, "Prereqs" refers to quantities that must be finite-difference to construct the RHSs.
Prereqs_to_print = [\
                   lhrh(lhs=gri.gfaccess("out_gfs","AevolParen"),rhs=gho.AevolParen),\
                   lhrh(lhs=gri.gfaccess("out_gfs","PevolParenU0"),rhs=gho.PevolParenU[0]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","PevolParenU1"),rhs=gho.PevolParenU[1]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","PevolParenU2"),rhs=gho.PevolParenU[2]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD00"),rhs=gho.SevolParenUD[0][0]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD01"),rhs=gho.SevolParenUD[0][1]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD02"),rhs=gho.SevolParenUD[0][2]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD10"),rhs=gho.SevolParenUD[1][0]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD11"),rhs=gho.SevolParenUD[1][1]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD12"),rhs=gho.SevolParenUD[1][2]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD20"),rhs=gho.SevolParenUD[2][0]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD21"),rhs=gho.SevolParenUD[2][1]),\
                   lhrh(lhs=gri.gfaccess("out_gfs","SevolParenUD22"),rhs=gho.SevolParenUD[2][2]),\
                   ]

metric_quantities_to_print = [\
                              lhrh(lhs=gri.gfaccess("out_gfs","gammaUU00"),rhs=gho.gammaUU[0][0]),\
                              lhrh(lhs=gri.gfaccess("out_gfs","gammaUU01"),rhs=gho.gammaUU[0][1]),\
                              lhrh(lhs=gri.gfaccess("out_gfs","gammaUU02"),rhs=gho.gammaUU[0][2]),\
                              lhrh(lhs=gri.gfaccess("out_gfs","gammaUU11"),rhs=gho.gammaUU[1][1]),\
                              lhrh(lhs=gri.gfaccess("out_gfs","gammaUU12"),rhs=gho.gammaUU[1][2]),\
                              lhrh(lhs=gri.gfaccess("out_gfs","gammaUU22"),rhs=gho.gammaUU[2][2]),\
                              lhrh(lhs=gri.gfaccess("out_gfs","gammadet"),rhs=gho.gammadet),\
                             ]

# Now, we'll add the Kreiss-Oliger dissipation terms to the RHS of StildeD
# thismodule = __name__
# diss_strength = par.Cparameters("REAL", thismodule, "diss_strength", 1e300) # diss_strength must be set in C, and
#                                                                             # we set it crazy high to ensure this.
# StildeD_dKOD = ixp.declarerank2("StildeD_dKOD","nosym")
# for k in range(DIM):
#     for i in range(DIM):
#         gho.Stilde_rhsD[i] += diss_strength * StildeD_dKOD[i][k]

# To best format this for the ETK, we'll need to register these gridfunctions.
#Stilde_rhsD = ixp.register_gridfunctions_for_single_rank1("AUX","Stilde_rhsD")
#A_rhsD = ixp.register_gridfunctions_for_single_rank1("AUX","A_rhsD")
#psi6Phi_rhs = gri.register_gridfunctions("AUX","psi6Phi_rhs")
Conservs_to_print = [\
                     lhrh(lhs=gri.gfaccess("rhs_gfs","StildeD0"),rhs=gho.Stilde_rhsD[0]),\
                     lhrh(lhs=gri.gfaccess("rhs_gfs","StildeD1"),rhs=gho.Stilde_rhsD[1]),\
                     lhrh(lhs=gri.gfaccess("rhs_gfs","StildeD2"),rhs=gho.Stilde_rhsD[2]),\
                     lhrh(lhs=gri.gfaccess("rhs_gfs","AD0"),rhs=gho.A_rhsD[0]),\
                     lhrh(lhs=gri.gfaccess("rhs_gfs","AD1"),rhs=gho.A_rhsD[1]),\
                     lhrh(lhs=gri.gfaccess("rhs_gfs","AD2"),rhs=gho.A_rhsD[2]),\
                     lhrh(lhs=gri.gfaccess("rhs_gfs","psi6Phi"),rhs=gho.psi6Phi_rhs),\
                    ]

import time
print("Generating C code for GiRaFFE RHSs in "+par.parval_from_str("reference_metric::CoordSystem")+" coordinates.")
start = time.time()

desc="Evaluate quantities to FD for RHSs"
name="quantities_to_FD_for_rhs_eval"

outCfunction(
    outfile  = os.path.join(Ccodesdir,"Prereqs.h"), desc=desc, name=name,
    params   = """const int Nxx_plus_2NGHOSTS[3],const REAL dxx[3], REAL *xx[3],
                  const REAL *in_gfs, REAL *aux_gfs""",
    body     = fin.FD_outputC("returnstring",Prereqs_to_print,params="outCverbose=False"),
    loopopts = "AllPoints,Enable_rfm_precompute")

desc="Calculate the metric determinant and inverse"
name="update_metric_det_inverse"

outCfunction(
    outfile  = os.path.join(Ccodesdir,"metric_quantities.h"), desc=desc, name=name,
    params   = """const int Nxx_plus_2NGHOSTS[3],const REAL dxx[3],REAL *xx[3],REAL *aux_gfs""",
    body     = fin.FD_outputC("returnstring",metric_quantities_to_print,params="outCverbose=False"),
    loopopts = "AllPoints,Enable_rfm_precompute")

desc="Evaluate the RHSs"
name="rhs_eval"

outCfunction(
    outfile  = os.path.join(Ccodesdir,"Conservs.h"), desc=desc, name=name,
    params   = """rfm_struct *restrict rfmstruct,const paramstruct *restrict params,
                  REAL *restrict in_gfs, REAL *restrict aux_gfs""",
    preloop  = """REAL invdx0 = 1.0/dxx[0];
                  REAL invdx1 = 1.0/dxx[1];
                  REAL invdx2 = 1.0/dxx[2];""",
    body     = fin.FD_outputC("returnstring",Conservs_to_print,params="outCverbose=False"),
    loopopts = "InteriorPoints,Enable_rfm_precompute",
    postloop = """LOOP_REGION(0,Nxx_plus_2NGHOSTS[0],0,Nxx_plus_2NGHOSTS[1],0,Nxx_plus_2NGHOSTS[2]){
                      if (sqrt(sq_radial_coord(xx[0][i0],xx[1][i1],xx[2][i2])) < min_radius_inside_of_which_conserv_to_prims_FFE_and_FFE_evolution_is_DISABLED){
                          idx = IDX3(i0,i1,i2);
                          rhs_gfs[IDX4pt(STILDED0GF, idx)] = 0.0;
                          rhs_gfs[IDX4pt(STILDED1GF, idx)] = 0.0;
                          rhs_gfs[IDX4pt(STILDED2GF, idx)] = 0.0;
                          rhs_gfs[IDX4pt(AD0GF, idx)] = 0.0;
                          rhs_gfs[IDX4pt(AD1GF, idx)] = 0.0;
                          rhs_gfs[IDX4pt(AD2GF, idx)] = 0.0;
                          rhs_gfs[IDX4pt(PSI6PHIGF, idx)] = 0.0;
                      }
                  }""")

end = time.time()

# Step 5: Import the function to calculate u0 and write it to a file.
import u0_smallb_Poynting__Cartesian.u0_smallb_Poynting__Cartesian as u0etc
#u0etc.compute_u0_smallb_Poynting__Cartesian(gammaDD,betaU,alpha,ValenciavU,BU)

with open(os.path.join(Ccodesdir,"computeu0_Cfunction.h"), "w") as file:
    file.write(u0etc.computeu0_Cfunction)


Generating C code for GiRaFFE RHSs in Cartesian coordinates.
Output C function quantities_to_FD_for_rhs_eval() to file GiRaFFE_standalone_Ccodes/Prereqs.h
Output C function update_metric_det_inverse() to file GiRaFFE_standalone_Ccodes/metric_quantities.h
Output C function rhs_eval() to file GiRaFFE_standalone_Ccodes/Conservs.h


<a id='cparams_rfm_and_domainsize'></a>

## Step 1.g: Output C codes needed for declaring and setting Cparameters; also set `free_parameters.h`
$$\label{cparams_rfm_and_domainsize}$$


In [8]:
# Step 3.d.i: Generate declare_Cparameters_struct.h, set_Cparameters_default.h, and set_Cparameters[-SIMD].h
par.generate_Cparameters_Ccodes(os.path.join(Ccodesdir))

# Step 3.d.ii: Set free_parameters.h
with open(os.path.join(Ccodesdir,"free_parameters.h"),"w") as file:
    file.write("""
// Step P3: Set free parameters
// Step P3a: Free parameters for the numerical grid
// Cartesian coordinates parameters
params.xmin = -4.0;params.xmax=4.0;
params.ymin = -4.0;params.ymax=4.0;
params.zmin = -4.0;params.zmax=4.0;
/*params.ymin = -0.0125;params.ymax=0.0125;
params.zmin = -0.0125;params.zmax=0.0125;*/

// Step P3b: Free parameters for the spacetime evolution
params.B_p_aligned_rotator                                                           = 1.0e-5;
params.Omega_aligned_rotator                                                         = 0.2;
// Disable these when doing 1D tests!
params.min_radius_inside_of_which_conserv_to_prims_FFE_and_FFE_evolution_is_DISABLED = -1.0; // Must be equal! v
params.R_NS_aligned_rotator                                                          = -1.0; // Must be equal! ^
params.xi                                                                            = 0.1;
params.diss_strength                                                                 = 0.3;
params.GAMMA_SPEED_LIMIT                                                             = 2000.0;
params.current_sheet_null_v = 0; // Boolean: 1=true,0=false

// Step P3c: Free parameters defining a 1D wave
//const REAL mu_AW  = -0.5; // The wave speed of the Alfven wave
#define mu_AW -0.5
params.lbound = -0.1*sqrt(1-mu_AW*mu_AW); // The left -most edge of the wave: divide by the
params.rbound =  0.1*sqrt(1-mu_AW*mu_AW); // The right-most edge of the wave: Lorentz Factor

// Time coordinate parameters
params.t_final = 2.0;  /* Final time is set so that at t=t_final,
                            * data at the origin have not been corrupted
                            * by the approximate outer boundary condition */
params.CFL_FACTOR = 0.5; // Set the CFL Factor

""")


## Step 4: Apply singular, curvilinear coordinate boundary conditions [as documented in the corresponding NRPy+ tutorial notebook](Tutorial-Start_to_Finish-Curvilinear_BCs.ipynb)

In [9]:
import CurviBoundaryConditions.CurviBoundaryConditions as cbcs
cmd.mkdir(os.path.join(Ccodesdir,"boundary_conditions/"))
cbcs.Set_up_CurviBoundaryConditions(os.path.join(Ccodesdir,"boundary_conditions/"))

Wrote to file "GiRaFFE_standalone_Ccodes/boundary_conditions/parity_conditions_symbolic_dot_products.h"
Evolved gridfunction "AD0" has parity type 1.
Evolved gridfunction "AD1" has parity type 2.
Evolved gridfunction "AD2" has parity type 3.
Evolved gridfunction "StildeD0" has parity type 1.
Evolved gridfunction "StildeD1" has parity type 2.
Evolved gridfunction "StildeD2" has parity type 3.
Evolved gridfunction "psi6Phi" has parity type 0.
Auxiliary gridfunction "AevolParen" has parity type 0.
Auxiliary gridfunction "BU0" has parity type 1.
Auxiliary gridfunction "BU1" has parity type 2.
Auxiliary gridfunction "BU2" has parity type 3.
Auxiliary gridfunction "PevolParenU0" has parity type 1.
Auxiliary gridfunction "PevolParenU1" has parity type 2.
Auxiliary gridfunction "PevolParenU2" has parity type 3.
Auxiliary gridfunction "SevolParenUD00" has parity type 4.
Auxiliary gridfunction "SevolParenUD01" has parity type 5.
Auxiliary gridfunction "SevolParenUD02" has parity type 6.
Auxiliar

## Step 3B: 
Now, we will generate the header files used to calculate the magnetic field $B^i$ from the vector potential $A_i$. We can use the function `GiRaFFE_HO_A2B()` that we wrote for this.

In [10]:
gri.glb_gridfcs_list = []
import GiRaFFE_HO.GiRaFFE_HO_A2B as A2B
A2B.GiRaFFE_HO_A2B("GiRaFFE_standalone_Ccodes/A2B/")

# Declaring StildeD as a gridfunction is unnecessary in GiRaFFE_HO. While it was declared in GiRaFFEfood_HO,
# those have since been cleared to avoid conflict; so, we re-declare it here.
StildeD = ixp.register_gridfunctions_for_single_rank1("EVOL","StildeD")


Wrote to file "GiRaFFE_standalone_Ccodes/A2B/B_from_A_order10.h"
Wrote to file "GiRaFFE_standalone_Ccodes/A2B/B_from_A_order8.h"
Wrote to file "GiRaFFE_standalone_Ccodes/A2B/B_from_A_order6.h"
Wrote to file "GiRaFFE_standalone_Ccodes/A2B/B_from_A_order4.h"
Wrote to file "GiRaFFE_standalone_Ccodes/A2B/B_from_A_order2.h"
Wrote to file "GiRaFFE_standalone_Ccodes/A2B/B_from_A_order2_dirx0_dnwind.h"
Wrote to file "GiRaFFE_standalone_Ccodes/A2B/B_from_A_order2_dirx0_upwind.h"
Wrote to file "GiRaFFE_standalone_Ccodes/A2B/B_from_A_order2_dirx1_dnwind.h"
Wrote to file "GiRaFFE_standalone_Ccodes/A2B/B_from_A_order2_dirx1_upwind.h"
Wrote to file "GiRaFFE_standalone_Ccodes/A2B/B_from_A_order2_dirx2_dnwind.h"
Wrote to file "GiRaFFE_standalone_Ccodes/A2B/B_from_A_order2_dirx2_upwind.h"


<a id='mainc'></a>

# Step 2: GiRaFFE_standalone.c: The Main C Code \[Back to [top](#toc)\]
$$\label{mainc}$$

In [11]:
# Part P0: Set the number of ghost cells, from NRPy+'s FD_CENTDERIVS_ORDER
with open(os.path.join(Ccodesdir,"NGHOSTS.h"), "w") as file:
    file.write("// Part P0: Set the number of ghost zones, from NRPy+'s FD_CENTDERIVS_ORDER\n")
    # We do not need upwinding in GiRaFFE
    file.write("#define NGHOSTS "+str(int(par.parval_from_str("finite_difference::FD_CENTDERIVS_ORDER")/2+1))+"\n")

<a id='import_headers'></a>

# Step 2.a: Import needed header files \[Back to [top](#toc)\]
$$\label{import_headers}$$

In [12]:
%%writefile $Ccodesdir/GiRaFFE_standalone.c
// Step P1: Import needed header files
#include "stdio.h"
#include "stdlib.h"
#include "math.h"
#include "string.h" // Needed for strncmp, etc.
#include "stdint.h" // Needed for Windows GCC 6.x compatibility
#ifndef M_PI
#define M_PI 3.141592653589793238462643383279502884L
#endif
#ifndef M_SQRT1_2
#define M_SQRT1_2 0.707106781186547524400844362104849039L
#endif

#include "time.h"
#include "NGHOSTS.h" // A NRPy+-generated file, which is set based on FD_CENTDERIVS_ORDER.


Writing GiRaFFE_standalone_Ccodes//GiRaFFE_standalone.c


<a id='data_type'></a>

# Step 2.b: Set data type \[Back to [top](#toc)\]
$$\label{data_type}$$

In [13]:
%%writefile -a $Ccodesdir/GiRaFFE_standalone.c

// Step P2: Add needed #define's to set data type, the IDX4() macro, and the gridfunctions
// Step P2a: set REAL=double, so that all floating point numbers are stored to at least ~16 significant digits.
#define REAL double


Appending to GiRaFFE_standalone_Ccodes//GiRaFFE_standalone.c


<a id='free_params'></a>

# Step 2.c: Set free parameters \[Back to [top](#toc)\]
$$\label{free_params}$$

In [14]:
%%writefile -a $Ccodesdir/GiRaFFE_standalone.c

#include "declare_Cparameters_struct.h"

Appending to GiRaFFE_standalone_Ccodes//GiRaFFE_standalone.c


<a id='idx4'></a>

# Step 2.d: Declare the IDX4 macro \[Back to [top](#toc)\]
$$\label{idx4}$$

In [15]:
%%writefile -a $Ccodesdir/GiRaFFE_standalone.c

// Step P6: Declare the IDX4(gf,i,j,k) macro, which enables us to store 4-dimensions of
//          data in a 1D array. In this case, consecutive values of "i"
//          (all other indices held to a fixed value) are consecutive in memory, where
//          consecutive values of "j" (fixing all other indices) are separated by
//          Nxx_plus_2NGHOSTS[0] elements in memory. Similarly, consecutive values of
//          "k" are separated by Nxx_plus_2NGHOSTS[0]*Nxx_plus_2NGHOSTS[1] in memory, etc.
#define IDX4S(g,i,j,k) \
( (i) + Nxx_plus_2NGHOSTS0 * ( (j) + Nxx_plus_2NGHOSTS1 * ( (k) + Nxx_plus_2NGHOSTS2 * (g) ) ) )
#define IDX4ptS(g,idx) ( (idx) + (Nxx_plus_2NGHOSTS0*Nxx_plus_2NGHOSTS1*Nxx_plus_2NGHOSTS2) * (g) )
#define IDX3(i,j,k) ( (i) + Nxx_plus_2NGHOSTS[0] * ( (j) + Nxx_plus_2NGHOSTS[1] * (k) ) )
// Assuming idx = IDX3(i,j,k). Much faster if idx can be reused over and over:
// To be deprecated soon:
#define IDX4(g,i,j,k) \
( (i) + Nxx_plus_2NGHOSTS[0] * ( (j) + Nxx_plus_2NGHOSTS[1] * ( (k) + Nxx_plus_2NGHOSTS[2] * (g) ) ) )
#define IDX4pt(g,idx)   ( (idx) + (Nxx_plus_2NGHOSTS[0]*Nxx_plus_2NGHOSTS[1]*Nxx_plus_2NGHOSTS[2]) * (g) )


Appending to GiRaFFE_standalone_Ccodes//GiRaFFE_standalone.c


<a id='gridfuncs'></a>

# Step 2.e: Define gridfunctions \[Back to [top](#toc)\]
$$\label{gridfuncs}$$

In [16]:
%%writefile -a $Ccodesdir/GiRaFFE_standalone.c

// Step P7: Set #define's for GRFFE gridfunctions. C code generated above
#include "boundary_conditions/gridfunction_defines.h"

#define LOOP_REGION(i0min,i0max, i1min,i1max, i2min,i2max) \
  for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++)
#define LOOP_ALL_GFS_GPS(ii) _Pragma("omp parallel for") \
  for(int (ii)=0;(ii)<Nxx_plus_2NGHOSTS_tot*NUM_EVOL_GFS;(ii)++)

#include "boundary_conditions/EigenCoord_xx_to_Cart.h"


Appending to GiRaFFE_standalone_Ccodes//GiRaFFE_standalone.c


<a id='bcs'></a>

# Step 2.f: Boundary Conditions, the A-to-B driver, and the conservative-to-primitive solver \[Back to [top](#toc)\]
$$\label{bcs}$$

In [17]:
%%writefile -a $Ccodesdir/GiRaFFE_standalone.c

// Step P8: Include basic functions needed to impose boundary conditions.
//#include "../CurviBoundaryConditions/curvilinear_parity_and_outer_boundary_conditions.h"
#include "boundary_conditions/GiRaFFE_boundary_conditions.h"

// Step P8c: Import C files for the A-to-B driver and the conservative-to-primitive solver
#include "A2B/driver_AtoB.c"
#include "driver_conserv_to_prims_FFE.C"


Appending to GiRaFFE_standalone_Ccodes//GiRaFFE_standalone.c


<a id='timestep'></a>

# Step 2.g: Find the CFL-constrained timestep \[Back to [top](#toc)\]
$$\label{timestep}$$

In [18]:
%%writefile -a $Ccodesdir/GiRaFFE_standalone.c

// Step P9: Find the CFL-constrained timestep
REAL find_timestep(const int Nxx_plus_2NGHOSTS[3],const REAL dxx[3],REAL *xx[3], const REAL CFL_FACTOR) {
  const REAL dxx0 = dxx[0], dxx1 = dxx[1], dxx2 = dxx[2];
  REAL dsmin = 1e38; // Start with a crazy high value... close to the largest number in single precision.
  LOOP_REGION(NGHOSTS,Nxx_plus_2NGHOSTS[0]-NGHOSTS, NGHOSTS,Nxx_plus_2NGHOSTS[1]-NGHOSTS, NGHOSTS,Nxx_plus_2NGHOSTS[2]-NGHOSTS) {
    const REAL xx0 = xx[0][i0], xx1 = xx[1][i1], xx2 = xx[2][i2];
    REAL ds_dirn0, ds_dirn1, ds_dirn2;
#include "ds_dirn.h"
//#define MIN(A, B) ( ((A) < (B)) ? (A) : (B) ) // Provided by driver_conserv_to_prims_FFE.C
    // Set dsmin = MIN(dsmin, ds_dirn0, ds_dirn1, ds_dirn2);
    dsmin = MIN(dsmin,MIN(ds_dirn0,MIN(ds_dirn1,ds_dirn2)));
  }
  return dsmin*CFL_FACTOR;
}


Appending to GiRaFFE_standalone_Ccodes//GiRaFFE_standalone.c


<a id='initial_data'></a>

# Step 2.h: Declare the function for the exact solution \[Back to [top](#toc)\]
$$\label{initial_data_c}$$

In [19]:
%%writefile -a $Ccodesdir/GiRaFFE_standalone.c

// Step P10: Declare the function for the exact solution. time==0 corresponds to the initial data.
void initial_data(const int Nxx_plus_2NGHOSTS[3],REAL *xx[3], REAL *out_gfs, REAL *aux_gfs) {
#pragma omp parallel for
  LOOP_REGION(0,Nxx_plus_2NGHOSTS[0], 0,Nxx_plus_2NGHOSTS[1], 0,Nxx_plus_2NGHOSTS[2]) {
    const int idx = IDX3(i0,i1,i2);
    aux_gfs[IDX4pt(GAMMADD00GF, idx)] = 1.0;
    aux_gfs[IDX4pt(GAMMADD01GF, idx)] = 0.0;
    aux_gfs[IDX4pt(GAMMADD02GF, idx)] = 0.0;
    aux_gfs[IDX4pt(GAMMADD11GF, idx)] = 1.0;
    aux_gfs[IDX4pt(GAMMADD12GF, idx)] = 0.0;
    aux_gfs[IDX4pt(GAMMADD22GF, idx)] = 1.0;
    aux_gfs[IDX4pt(BETAU0GF, idx)] = 0.0;
    aux_gfs[IDX4pt(BETAU1GF, idx)] = 0.0;
    aux_gfs[IDX4pt(BETAU2GF, idx)] = 0.0;
    aux_gfs[IDX4pt(ALPHAGF, idx)] = 1.0;
    REAL xx0 = xx[0][i0];
    REAL xx1 = xx[1][i1];
    REAL xx2 = xx[2][i2];
    if(xx0<=lbound) {
#include "GiRaFFEfood_A_v_1D_tests_left.h"
    }
    else if (xx0<rbound) {
#include "GiRaFFEfood_A_v_1D_tests_center.h"
    }
    else {
#include "GiRaFFEfood_A_v_1D_tests_right.h"
    }
    out_gfs[IDX4pt(PSI6PHIGF, idx)] = 0.0;
  }
}

void initial_Stilde_from_ID(const int Nxx_plus_2NGHOSTS[3],REAL *xx[3],const REAL *aux_gfs, REAL *out_gfs) {
    LOOP_REGION(0,Nxx_plus_2NGHOSTS[0],0,Nxx_plus_2NGHOSTS[1],0,Nxx_plus_2NGHOSTS[2]){
#include "GiRaFFEfood_HO_Stilde.h"
    }
}


Appending to GiRaFFE_standalone_Ccodes//GiRaFFE_standalone.c


<a id='rhsC'></a>

# Step 2.i: Declare the functions to evaluate the GRFFE RHSs \[Back to [top](#toc)\]
$$\label{rhsC}$$

In [20]:
%%writefile -a $Ccodesdir/GiRaFFE_standalone.c

// Step P11: Declare the functions to evaluate the GRFFE RHSs
// Step P11a: Create the function to calculate u4upperZero:
void calc_u0(const int Nxx_plus_2NGHOSTS[3],REAL *aux_gfs)
{
    int idx;
    LOOP_REGION(0,Nxx_plus_2NGHOSTS[0],0,Nxx_plus_2NGHOSTS[1],0,Nxx_plus_2NGHOSTS[2]){
        idx = IDX3(i0,i1,i2);
        REAL u0;
        REAL ValenciavU0 = aux_gfs[IDX4pt(VALENCIAVU0GF,idx)];
        REAL ValenciavU1 = aux_gfs[IDX4pt(VALENCIAVU1GF,idx)];
        REAL ValenciavU2 = aux_gfs[IDX4pt(VALENCIAVU2GF,idx)];
        REAL alpha = aux_gfs[IDX4pt(ALPHAGF,idx)];
        REAL gammaDD00 = aux_gfs[IDX4pt(GAMMADD00GF,idx)];
        REAL gammaDD01 = aux_gfs[IDX4pt(GAMMADD01GF,idx)];
        REAL gammaDD02 = aux_gfs[IDX4pt(GAMMADD02GF,idx)];
        REAL gammaDD11 = aux_gfs[IDX4pt(GAMMADD11GF,idx)];
        REAL gammaDD12 = aux_gfs[IDX4pt(GAMMADD12GF,idx)];
        REAL gammaDD22 = aux_gfs[IDX4pt(GAMMADD22GF,idx)];

        #include "computeu0_Cfunction.h"

        aux_gfs[IDX4pt(U4UPPERZEROGF,idx)] = u0;
        aux_gfs[IDX4pt(VALENCIAVU0GF,idx)] = ValenciavU0;
        aux_gfs[IDX4pt(VALENCIAVU1GF,idx)] = ValenciavU1;
        aux_gfs[IDX4pt(VALENCIAVU2GF,idx)] = ValenciavU2;
    }
}

// Step P11b: Set the quantities to be differentiated by finite difference for the RHSs--ALWAYS run immediately
//            before rhs_eval()
#include "Prereqs.h"

// While this code is generally cartesian, we will need an r coordinate for the evolution:
REAL sq_radial_coord(const REAL x,const REAL y,const REAL z) { return x*x+y*y+z*z; }

// Step P11c: Set the RHSs themselves.
#include "Conservs.h"


Appending to GiRaFFE_standalone_Ccodes//GiRaFFE_standalone.c


<a id='main'></a>

# Step 2.j: The `main()` function \[Back to [top](#toc)\]
$$\label{main}$$

In [21]:
%%writefile -a $Ccodesdir/GiRaFFE_standalone.c

// main() function:
// Step 0: Read command-line input, set up grid structure, allocate memory for gridfunctions, set up coordinates
// Step 1: Set up scalar wave initial data
// Step 2: Evolve scalar wave initial data forward in time using Method of Lines with RK4 algorithm,
//         applying quadratic extrapolation outer boundary conditions.
// Step 3: Output relative error between numerical and exact solution.
// Step 4: Free all allocated memory
int main(int argc, const char *argv[]) {
    // Step 0a: Read command-line input, error out if nonconformant
    if((argc != 4 && argc != 5) || atoi(argv[1]) < NGHOSTS || atoi(argv[2]) < NGHOSTS || atoi(argv[3]) < 2 /* FIXME; allow for axisymmetric sims */) {
        fprintf(stderr,"Error: Expected three command-line arguments: ./BrillLindquist_Playground Nx0 Nx1 Nx2,\n");
        fprintf(stderr,"where Nx[0,1,2] is the number of grid points in the 0, 1, and 2 directions.\n");
        fprintf(stderr,"Nx[] MUST BE larger than NGHOSTS (= %d)\n",NGHOSTS);
        exit(1);
    }
    if(argc == 5) {
        CFL_FACTOR = strtod(argv[4],NULL);
        if(CFL_FACTOR > 0.5 && atoi(argv[3])!=2) {
            fprintf(stderr,"WARNING: CFL_FACTOR was set to %e, which is > 0.5.\n",CFL_FACTOR);
            fprintf(stderr,"         This will generally only be stable if the simulation is purely axisymmetric\n");
            fprintf(stderr,"         However, Nx2 was set to %d>2, which implies a non-axisymmetric simulation\n",atoi(argv[3]));
        }
    }
    // Step 0b: Set up numerical grid structure, first in space...
    const int Nxx[3] = { atoi(argv[1]), atoi(argv[2]), atoi(argv[3]) };
    if(Nxx[0]%2 != 0 || Nxx[1]%2 != 0 || Nxx[2]%2 != 0) {
        fprintf(stderr,"Error: Cannot guarantee a proper cell-centered grid if number of grid cells not set to even number.\n");
        fprintf(stderr,"       For example, in case of angular directions, proper symmetry zones will not exist.\n");
        exit(1);
    }
    const int Nxx_plus_2NGHOSTS[3] = { Nxx[0]+2*NGHOSTS, Nxx[1]+2*NGHOSTS, Nxx[2]+2*NGHOSTS };
    const int Nxx_plus_2NGHOSTS_tot = Nxx_plus_2NGHOSTS[0]*Nxx_plus_2NGHOSTS[1]*Nxx_plus_2NGHOSTS[2];
#include "xxminmax.h"

    // Step 0c: Allocate memory for gridfunctions
    REAL *aux_gfs  = (REAL *)malloc(sizeof(REAL) * NUM_AUX_GFS * Nxx_plus_2NGHOSTS_tot);
#include "../MoLtimestepping/RK_Allocate_Memory.h"
    for(int i=0;i<NUM_EVOL_GFS * Nxx_plus_2NGHOSTS_tot;i++) {
        y_n_gfs[i] = 1.0/0.0;
        //k_even_gfs[i] = 1.0/0.0;
        //k_odd_gfs[i] = 1.0/0.0;
    }
    for(int i=0;i<NUM_AUX_GFS * Nxx_plus_2NGHOSTS_tot;i++) {
        aux_gfs[i] = 1.0/0.0;
    }
    REAL *evol_gfs_exact = (REAL *)malloc(sizeof(REAL) * NUM_EVOL_GFS * Nxx_plus_2NGHOSTS_tot);
    REAL *aux_gfs_exact  = (REAL *)malloc(sizeof(REAL) * NUM_AUX_GFS * Nxx_plus_2NGHOSTS_tot);

    // Step 0c: Set free parameters, overwriting Cparameters defaults
    //          by hand or with command-line input, as desired.
#include "free_parameters.h"
#include "set_Cparameters-nopointer.h"

    // Step 0d: Set up space and time coordinates
    // Step 0d.i: Set \Delta x^i on uniform grids.
    REAL dxx[3];
    for(int i=0;i<3;i++) dxx[i] = (xxmax[i] - xxmin[i]) / ((REAL)Nxx[i]);
    for(int i=0;i<3;i++) printf("dxx[%d] = %.15e\n",i,dxx[i]);

    // Step 0d.ii: Set up uniform coordinate grids
    REAL *xx[3];
    for(int i=0;i<3;i++) {
        xx[i] = (REAL *)malloc(sizeof(REAL)*Nxx_plus_2NGHOSTS[i]);
        for(int j=0;j<Nxx_plus_2NGHOSTS[i];j++) {
            xx[i][j] = xxmin[i] + ((REAL)(j-NGHOSTS) + (1.0/2.0))*dxx[i]; // Cell-centered grid.
        }
    }

    // Step 0d.iii: Set timestep based on smallest proper distance between gridpoints and CFL factor
    REAL dt = find_timestep(Nxx_plus_2NGHOSTS, dxx,xx, CFL_FACTOR);
    printf("# Timestep set to = %e\n",(double)dt);
    int N_final = (int)(t_final / dt + 0.5); // The number of iterations in time.
                                           //Add 0.5 to account for C rounding down integers.

    // Step 1: Set up initial data to an exact solution at time=0:
    // Step 1a: Set up the exact initial data:
    initial_data(Nxx_plus_2NGHOSTS, xx, y_n_gfs, aux_gfs);

    // Step 1b: Run the initial A-to-B driver:
    driver_A_to_B(Nxx, Nxx_plus_2NGHOSTS, dxx, y_n_gfs, aux_gfs);

    // Step 1c: Solve for StildeD from BU and ValenciavU
    initial_Stilde_from_ID(Nxx_plus_2NGHOSTS, xx, aux_gfs, y_n_gfs);

    // Step 1d: Apply boundary conditions, as initial data
    //          are sometimes ill-defined in ghost zones.
    //          E.g., spherical initial data might not be
    //          properly defined at points where r=-1.
    // Step 1e: Run the conservative-to-primitive solver:
    GiRaFFE_HO_conserv_to_prims_FFE(Nxx, Nxx_plus_2NGHOSTS, dxx,xx, y_n_gfs, aux_gfs);

    //apply_bcs(Nxx, Nxx_plus_2NGHOSTS, bc_gz_map,bc_parity_conditions,NUM_EVOL_GFS,evol_gf_parity, y_n_gfs);
    apply_bcs(Nxx, Nxx_plus_2NGHOSTS, y_n_gfs, aux_gfs);

    // Rerun AtoB for consistency:
    driver_A_to_B(Nxx, Nxx_plus_2NGHOSTS, dxx, y_n_gfs, aux_gfs);

    // Step 3: Start the timer, for keeping track of how fast the simulation is progressing.
    struct timespec start, end;
    clock_gettime(CLOCK_REALTIME, &start);

    // Step 4: Integrate the initial data forward in time using the Method of Lines and RK4
    for(int n=0;n<=N_final;n++) { // Main loop to progress forward in time.

    /* Step 5: Output 2D data file, for visualization. Do this first to get initial data. */
    // For convergence testing, we'll shift the grid x -> x-1 and output initial data again, giving the exact solution.
    LOOP_REGION(0,Nxx_plus_2NGHOSTS[0],0,1,0,1) {
        xx[0][i0] += -mu_AW*(n)*dt;
    }
    // Recalculate the initial data on the shifted grid, using the same process as before for consistency.
    initial_data(Nxx_plus_2NGHOSTS, xx, evol_gfs_exact, aux_gfs_exact);
    driver_A_to_B(Nxx, Nxx_plus_2NGHOSTS, dxx, evol_gfs_exact, aux_gfs_exact);
    initial_Stilde_from_ID(Nxx_plus_2NGHOSTS, xx, aux_gfs_exact, evol_gfs_exact);
    GiRaFFE_HO_conserv_to_prims_FFE(Nxx, Nxx_plus_2NGHOSTS, dxx,xx, evol_gfs_exact, aux_gfs_exact);
    driver_A_to_B(Nxx, Nxx_plus_2NGHOSTS, dxx, evol_gfs_exact, aux_gfs_exact);
    apply_bcs(Nxx, Nxx_plus_2NGHOSTS, evol_gfs_exact, aux_gfs_exact);
    // And now, we'll set the grid back to rights.
    LOOP_REGION(0,Nxx_plus_2NGHOSTS[0],0,1,0,1) {
        xx[0][i0] -= -mu_AW*(n)*dt;
    }

    if(n%10 == 0) {
        printf("Writing output...\n");
//        const int i1mid = Nxx_plus_2NGHOSTS[1]/2;
        char filename[100];
        sprintf(filename,"out%d-%08d_numer.txt",Nxx[0],n);
        FILE *out2D_numer = fopen(filename, "w");
        //LOOP_REGION(NGHOSTS,Nxx_plus_2NGHOSTS[0]-NGHOSTS, NGHOSTS,Nxx_plus_2NGHOSTS[1]-NGHOSTS, NGHOSTS,Nxx_plus_2NGHOSTS[2]-NGHOSTS) {
        //LOOP_REGION(0,Nxx_plus_2NGHOSTS[0], Nxx_plus_2NGHOSTS[1]/2,Nxx_plus_2NGHOSTS[1]/2+1,Nxx_plus_2NGHOSTS[2]/2,Nxx_plus_2NGHOSTS[2]/2+1) {
        LOOP_REGION(0,Nxx_plus_2NGHOSTS[0],Nxx_plus_2NGHOSTS[1]/2,Nxx_plus_2NGHOSTS[1]/2+1,Nxx_plus_2NGHOSTS[2]/2,Nxx_plus_2NGHOSTS[2]/2+1) {
            const int idx = IDX3(i0,i1,i2);
            REAL xx0 = xx[0][i0];
            REAL xx1 = xx[1][i1];
            REAL xx2 = xx[2][i2];
            REAL xCart[3];
#include "xx_to_Cart.h"
            fprintf(out2D_numer,"%.16e %.16e %.16e %.16e %.16e %.16e %.16e %.16e %.16e %.16e %.16e %.16e %.16e %.16e %.16e\n",
                    xCart[0],xCart[1],xCart[2],
                    aux_gfs[IDX4pt(BU0GF,idx)],aux_gfs[IDX4pt(BU1GF,idx)],aux_gfs[IDX4pt(BU2GF,idx)],
                    y_n_gfs[IDX4pt(AD0GF,idx)],y_n_gfs[IDX4pt(AD1GF,idx)],y_n_gfs[IDX4pt(AD2GF,idx)],
                    y_n_gfs[IDX4pt(STILDED0GF,idx)],y_n_gfs[IDX4pt(STILDED1GF,idx)],y_n_gfs[IDX4pt(STILDED2GF,idx)],
                    aux_gfs[IDX4pt(VALENCIAVU0GF,idx)],aux_gfs[IDX4pt(VALENCIAVU1GF,idx)],aux_gfs[IDX4pt(VALENCIAVU2GF,idx)]);
        }
        fclose(out2D_numer);

        // Now rerun the same output code we used in the main simulation.
        printf("Writing EXACT output...\n");
        sprintf(filename,"out%d-%08d_exact.txt",Nxx[0],n);
        FILE *out2D_exact = fopen(filename, "w");
        //LOOP_REGION(NGHOSTS,Nxx_plus_2NGHOSTS[0]-NGHOSTS, NGHOSTS,Nxx_plus_2NGHOSTS[1]-NGHOSTS, NGHOSTS,Nxx_plus_2NGHOSTS[2]-NGHOSTS) {
        LOOP_REGION(0,Nxx_plus_2NGHOSTS[0], Nxx_plus_2NGHOSTS[1]/2,Nxx_plus_2NGHOSTS[1]/2+1,Nxx_plus_2NGHOSTS[2]/2,Nxx_plus_2NGHOSTS[2]/2+1) {
            const int idx = IDX3(i0,i1,i2);
            REAL xx0 = xx[0][i0];
            REAL xx1 = xx[1][i1];
            REAL xx2 = xx[2][i2];
            REAL xCart[3];
#include "xx_to_Cart.h"
            fprintf(out2D_exact,"%.16e %.16e %.16e %.16e %.16e %.16e %.16e %.16e %.16e %.16e %.16e %.16e %.16e %.16e %.16e\n",
                    xCart[0],xCart[1],xCart[2],
                    aux_gfs_exact[IDX4pt(BU0GF,idx)],aux_gfs_exact[IDX4pt(BU1GF,idx)],aux_gfs_exact[IDX4pt(BU2GF,idx)],
                    evol_gfs_exact[IDX4pt(AD0GF,idx)],evol_gfs_exact[IDX4pt(AD1GF,idx)],evol_gfs_exact[IDX4pt(AD2GF,idx)],
                    evol_gfs_exact[IDX4pt(STILDED0GF,idx)],evol_gfs_exact[IDX4pt(STILDED1GF,idx)],evol_gfs_exact[IDX4pt(STILDED2GF,idx)],
                    aux_gfs_exact[IDX4pt(VALENCIAVU0GF,idx)],aux_gfs_exact[IDX4pt(VALENCIAVU1GF,idx)],aux_gfs_exact[IDX4pt(VALENCIAVU2GF,idx)]);
        }
        fclose(out2D_exact);
    }

    // Step 6: Step forward one timestep (t -> t+dt) in time using
    //           chosen RK-like MoL timestepping algorithm
#include "../MoLtimestepping/RK_MoL.h"


    for(int gf=0;gf<NUM_EVOL_GFS;gf++) {
        LOOP_REGION(NGHOSTS,Nxx_plus_2NGHOSTS[0]-NGHOSTS,NGHOSTS,Nxx_plus_2NGHOSTS[1]-NGHOSTS,NGHOSTS,Nxx_plus_2NGHOSTS[2]-NGHOSTS){
            if(isnan(y_n_gfs[IDX4(gf,i0,i1,i2)])) {
                printf("ERROR, FOUND A NAN ON GF %d AT POINT %d %d %d\n",gf,i0,i1,i2);
                exit(1);
            }
        }
    }

    // This function will now write the Exact solution for StildeD to the boundaries.
    //apply_bcs_EXACT_StildeD(Nxx, Nxx_plus_2NGHOSTS, xx,y_n_gfs,evol_gfs_exact);

    // Progress indicator printing to stdout
    // Measure average time per iteration
    clock_gettime(CLOCK_REALTIME, &end);
    const long long unsigned int time_in_ns = 1000000000L * (end.tv_sec - start.tv_sec) + end.tv_nsec - start.tv_nsec;
    const REAL s_per_iteration_avg = ((REAL)time_in_ns / (REAL)n) / 1.0e9;

    const int iterations_remaining = N_final - n;
    const REAL time_remaining_in_mins = s_per_iteration_avg * (REAL)iterations_remaining / 60.0;

    const REAL num_RHS_pt_evals = (REAL)(Nxx[0]*Nxx[1]*Nxx[2]) * 4.0 * (REAL)n; // 4 RHS evals per gridpoint for RK4
    const REAL RHS_pt_evals_per_sec = num_RHS_pt_evals / ((REAL)time_in_ns / 1.0e9);

    // Progress indicator printing to stderr
    fprintf(stderr,"%c[2K", 27); // Clear the line
    fprintf(stderr,"It: %d t=%.2f | %.1f%%; ETA %.0f s | t/h %.2f | gp/s %.2e\r",  // \r is carriage return, move cursor to the beginning of the line
           n, n * (double)dt, (double)(100.0 * (REAL)n / (REAL)N_final),
           (double)time_remaining_in_mins*60, (double)(dt * 3600.0 / s_per_iteration_avg), (double)RHS_pt_evals_per_sec);
    fflush(stderr); // Flush the stderr buffer
    } // End main loop to progress forward in time.

    fprintf(stderr,"\n"); // Clear the line.

    /* Step 6: Free all allocated memory */
#include "../MoLtimestepping/RK_Free_Memory.h"
    free(aux_gfs);
    free(aux_gfs_exact);
    free(evol_gfs_exact);
    for(int i=0;i<3;i++) free(xx[i]);
    return 0;
}

Appending to GiRaFFE_standalone_Ccodes//GiRaFFE_standalone.c


Now that the C code is put together, we will compile and run the code.

In [22]:
import cmdline_helper as cmd

print("Now compiling, should take ~2 seconds...\n")
start = time.time()
cmd.C_compile(os.path.join(Ccodesdir,"GiRaFFE_standalone.c"), "GiRaFFE_standalone_1D")
# Switching back so I can use a nan checker
#!gcc -g -O2 -fopenmp GiRaFFE_standalone/GiRaFFE_standalone.c -o GiRaFFE_standalone_1D -lm
end = time.time()
print("Finished in "+str(end-start)+" seconds.\n\n")

cmd.delete_existing_files("out128*.txt")
cmd.delete_existing_files("out2560*.txt")

print("Now running at low resolution. Should take ~20 seconds...\n")
start = time.time()
#cmd.Execute("GiRaFFE_standalone_1D", "1280 2 2 0.5")
# Switching back to see more output
!taskset -c 0,1 ./GiRaFFE_standalone_1D 1280 2 2 0.5
end = time.time()
print("Finished in "+str(end-start)+" seconds.\n\n")

print("Now running at medium resolution. Should take ~300 seconds...\n")
start = time.time()
#cmd.Execute("GiRaFFE_standalone_1D", "2560 8 8 0.5")
end = time.time()
print("Finished in "+str(end-start)+" seconds.\n\n")

Now compiling, should take ~2 seconds...

Compiling executable...
Executing `gcc -Ofast -fopenmp -march=native -funroll-loops GiRaFFE_standalone_Ccodes/GiRaFFE_standalone.c -o GiRaFFE_standalone_1D -lm`...
In file included from GiRaFFE_standalone_Ccodes/GiRaFFE_standalone.c:51:0:
GiRaFFE_standalone_Ccodes/boundary_conditions/GiRaFFE_boundary_conditions.h: In function ‘apply_bcs_EXACT’:
GiRaFFE_standalone_Ccodes/boundary_conditions/GiRaFFE_boundary_conditions.h:146:25: error: ‘Nxx’ undeclared (first use in this function); did you mean ‘Nxx2’?
       FACE_UPDATE_EXACT(Nxx,Nxx_plus_2NGHOSTS,xx,n,dt,out_gfs,aux_gfs,imin[0]-1,imin[0], imin[1],imax[1], imin[2],imax[2], MINFACE,NUL,NUL);
                         ^~~
                         Nxx2
GiRaFFE_standalone_Ccodes/boundary_conditions/GiRaFFE_boundary_conditions.h:146:25: note: each undeclared identifier is reported only once for each function it appears in
GiRaFFE_standalone_Ccodes/boundary_conditions/GiRaFFE_boundary_conditions.h:146:

In file included from GiRaFFE_standalone_Ccodes/GiRaFFE_standalone.c:51:0:
GiRaFFE_standalone_Ccodes/boundary_conditions/GiRaFFE_boundary_conditions.h: In function ‘apply_bcs_EXACT’:
GiRaFFE_standalone_Ccodes/boundary_conditions/GiRaFFE_boundary_conditions.h:146:25: error: ‘Nxx’ undeclared (first use in this function); did you mean ‘Nxx2’?
       FACE_UPDATE_EXACT(Nxx,Nxx_plus_2NGHOSTS,xx,n,dt,out_gfs,aux_gfs,imin[0]-1,imin[0], imin[1],imax[1], imin[2],imax[2], MINFACE,NUL,NUL);
                         ^~~
                         Nxx2
GiRaFFE_standalone_Ccodes/boundary_conditions/GiRaFFE_boundary_conditions.h:146:25: note: each undeclared identifier is reported only once for each function it appears in
GiRaFFE_standalone_Ccodes/boundary_conditions/GiRaFFE_boundary_conditions.h:146:29: error: ‘Nxx_plus_2NGHOSTS’ undeclared (first use in this function); did you mean ‘Nxx_plus_2NGHOSTS2’?
       FACE_UPDATE_EXACT(Nxx,Nxx_plus_2NGHOSTS,xx,n,dt,out_gfs,aux_gfs,imin[0]-1,imin[0], imin[1],i

In file included from GiRaFFE_standalone_Ccodes/GiRaFFE_standalone.c:51:0:
GiRaFFE_standalone_Ccodes/boundary_conditions/GiRaFFE_boundary_conditions.h: In function ‘apply_bcs_EXACT’:
GiRaFFE_standalone_Ccodes/boundary_conditions/GiRaFFE_boundary_conditions.h:146:25: error: ‘Nxx’ undeclared (first use in this function); did you mean ‘Nxx2’?
       FACE_UPDATE_EXACT(Nxx,Nxx_plus_2NGHOSTS,xx,n,dt,out_gfs,aux_gfs,imin[0]-1,imin[0], imin[1],imax[1], imin[2],imax[2], MINFACE,NUL,NUL);
                         ^~~
                         Nxx2
GiRaFFE_standalone_Ccodes/boundary_conditions/GiRaFFE_boundary_conditions.h:146:25: note: each undeclared identifier is reported only once for each function it appears in
GiRaFFE_standalone_Ccodes/boundary_conditions/GiRaFFE_boundary_conditions.h:146:29: error: ‘Nxx_plus_2NGHOSTS’ undeclared (first use in this function); did you mean ‘Nxx_plus_2NGHOSTS2’?
       FACE_UPDATE_EXACT(Nxx,Nxx_plus_2NGHOSTS,xx,n,dt,out_gfs,aux_gfs,imin[0]-1,imin[0], imin[1],i

SystemExit: 1

/home/penelson/jup38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


<a id='convergence'></a>

# Step 3: Code validation: Verify that relative error in numerical solution converges to zero at the expected order \[Back to [top](#toc)\]
$$\label{convergence}$$
Now, we will load the data generated by the simulation and plot it in order to test for convergence. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Data_numer = np.loadtxt("out256-00000010_numer.txt")
#Data_oldbc = np.loadtxt("oldbc_out1280-00000030_numer.txt")
#Data_num_2 = np.loadtxt("out2560-00000600_numer.txt")
Data_exact = np.loadtxt("out256-00000010_exact.txt")
#Data_exa_2 = np.loadtxt("out2560-00000600_exact.txt")

#path = "/home/penelson/OldCactus/Cactus/exe/ABE-GiRaFFEfood_1D_AlfvenWave/"
#Data_oldG = np.loadtxt(path + "giraffe-grmhd_primitives_bi.x.asc")
#n=32
#Data_oldG_atT = Data_oldG[1285*n:1285*(n+1)-1,:]


#predicted_order = 4.0
column = 5
plt.figure()
#plt.plot(Data_numer[:,0],Data_numer[:,column]-Data_oldbc[:,column],'.')
#plt.plot(Data_num_2[:,0],(Data_num_2[:,column]-Data_exa_2[:,column])*(2**predicted_order),'.')
plt.plot(Data_numer[:,0],Data_numer[:,column],'.')
plt.plot(Data_exact[:,0],Data_exact[:,column],'.')
#plt.plot(Data_oldG_atT[1:-1,9],)
#plt.plot(Data_numer[1:-2,1],(Data_numer[0:-3,column]-Data_numer[2:-1,column])/3.125e-3,'.-')
#plt.plot(Data_numer[1:-2,1],(Data_oldbc[0:-3,column]-Data_oldbc[2:-1,column])/3.125e-3,'.')
#plt.plot(Data_numer[0,0],Data_numer[0,column],'o')
#plt.xlim(-0.15,0.15)
#plt.ylim(-0.2e-10,0.2e-10)
plt.xlabel("y")
plt.ylabel("BU2")
plt.show()


This code will create an animation of the wave over time to hopefully show us where things go wrong.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import savefig
from IPython.display import HTML
import matplotlib.image as mgimg

import glob
import sys
from matplotlib import animation

globby = glob.glob('out1280-00*.txt')
file_list = []
for x in sorted(globby):
    file_list.append(x)

number_of_files = len(file_list)/2

for timestep in range(number_of_files):
    fig = plt.figure()
    numer_filename = file_list[2*timestep]
    exact_filename = file_list[2*timestep+1]
    Numer = np.loadtxt(numer_filename)
    Exact = np.loadtxt(exact_filename)

    plt.title("Alfven Wave")
    plt.xlabel("x")
    plt.ylabel("BU2")
    plt.xlim(-4.0,4.0)
    plt.ylim(1.14,1.57)

    plt.plot(Numer[:,0],Numer[:,5])
    plt.plot(Exact[:,0],Exact[:,5],'.')
    savefig(numer_filename+".png",dpi=150)
    plt.close(fig)
    sys.stdout.write("%c[2K" % 27)
    sys.stdout.write("Processing file "+numer_filename+"\r")
    sys.stdout.flush()

In [ ]:
## VISUALIZATION ANIMATION, PART 2: Combine PNGs to generate movie ##
# https://stackoverflow.com/questions/14908576/how-to-remove-frame-from-matplotlib-pyplot-figure-vs-matplotlib-figure-frame
# https://stackoverflow.com/questions/23176161/animating-pngs-in-matplotlib-using-artistanimation
!rm -f GiRaFFE_HO-1D_tests.mp4

fig = plt.figure(frameon=False)
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')

myimages = []

for i in range(len(file_list)/2):
    img = mgimg.imread(file_list[2*i]+".png")
    imgplot = plt.imshow(img)
    myimages.append([imgplot])

ani = animation.ArtistAnimation(fig, myimages, interval=100,  repeat_delay=1000)
plt.close()
ani.save('GiRaFFE_HO-1D_tests.mp4', fps=5,dpi=150)

In [ ]:
%%HTML
<video width="480" height="360" controls>
  <source src="GiRaFFE_HO-1D_tests.mp4" type="video/mp4">
</video>

<a id='latex_pdf_output'></a>

# Step 3: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-Start_to_Finish-GiRaFFE_HO-1D_tests.pdf](Tutorial-Start_to_Finish-GiRaFFE_HO-1D_tests.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [ ]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx --log-level='WARN' Tutorial-Start_to_Finish-GiRaFFE_HO-1D_tests.ipynb
!pdflatex -interaction=batchmode Tutorial-Start_to_Finish-GiRaFFE_HO-1D_tests.tex
!pdflatex -interaction=batchmode Tutorial-Start_to_Finish-GiRaFFE_HO-1D_tests.tex
!pdflatex -interaction=batchmode Tutorial-Start_to_Finish-GiRaFFE_HO-1D_tests.tex
!rm -f Tut*.out Tut*.aux Tut*.log